In [ ]:
import argparse
import json
import tensorflow as tf
import numpy as np
import h5py
import copy
import matplotlib.pyplot as plt
import matplotlib.path as mplPath

from fr3D.train.utils import setup_datasets
from fr3D.data.utils import get_normalization
from fr3D.models import ConvAutoencoder, ConvAutoencoderCGAN, ConvAutoencoderC

tf.keras.backend.set_image_data_format('channels_last')

def check_in_polygon(pts, vertices):
    path = mplPath.Path(vertices)
    return path.contains_points(pts)

def mape_with_threshold(yp, yt, pcterror_threshold=np.inf, max_magnitude_threshold=0.0, eps=1e-7):
    pct_errors = 100*tf.abs((yp-yt)/(eps + yt))
    pcterror_mask = pct_errors < pcterror_threshold
    max_magnitude_mask = tf.logical_not(tf.abs(yt) < (max_magnitude_threshold*tf.reduce_max(tf.abs(yt))))
    filtering_indices = tf.where(tf.logical_and(pcterror_mask, max_magnitude_mask))
    filtered_pcterrors = tf.gather_nd(pct_errors, filtering_indices)
    return float(tf.reduce_mean(filtered_pcterrors))

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
Re = 500
expt_variable = 'V'

#dataset_path = f'/fr3D/postprocessed/annulus_64_{Re}.h5'
dataset_path = f'/fr3D/postprocessed/annulus_64.h5'
#dataset_path = f'/fr3D/postprocessed/annulus_64_plane.h5'

#CGAN
#experiment_config = f'/fr3D/configs/training/ConvAutoencoderCGAN_{expt_variable}.json'
#weights_path = f'/storage/weights/ConvAutoencoderCGAN_{expt_variable}_Annulus64/ConvAutoencoderCGAN_{expt_variable}_Annulus64.h5'

#ConvAutoencoderC
experiment_config = f'/fr3D/configs/training/ConvAutoencoderC_{expt_variable}.json'
weights_path = f'/storage/weights{Re}/ConvAutoencoderC_{expt_variable}_Annulus64/ConvAutoencoderC_{expt_variable}_Annulus64.h5'



datasetf = h5py.File(dataset_path,'r')

shuf_buf = 1
config = json.load(open(experiment_config,'r'))
train_dataset, test_dataset = setup_datasets(config, dataset_path, shuf_buf, case_names=True, evaluation=True)
sensor_shape = train_dataset.element_spec[0][0].shape
full_field_shape = train_dataset.element_spec[0][1].shape

def get_normalization_type(node_configs):
    normalization_spec = {'method': None}
    for node in node_configs:
        if node['nodetype'] == 'normalize':
            normalization_spec = node['normalization_spec']
            break
    normalizer = get_normalization(**normalization_spec, batch_mode=True)
    return normalizer
normalizer = get_normalization_type(config['dataset']['node_configurations'])

In [ ]:
generator = ConvAutoencoder(input_shape=full_field_shape[1:], **config['generator'])
model = ConvAutoencoderCGAN(sensor_shape[1],generator,**config.get('discriminator',{}))
model.compile(d_optimizer = tf.keras.optimizers.get(config['training']['d_optimizer']),
            g_optimizer = tf.keras.optimizers.get(config['training']['g_optimizer']),
            l_optimizer = tf.keras.optimizers.get(config['training']['l_optimizer']),
            global_batch_size=config['dataset']['batch_size'])
_ = model(next(iter(test_dataset))[0][0], autoencode=False)

model.load_weights(weights_path)


In [ ]:
model = ConvAutoencoderC(dense_input_units=sensor_shape[1],
                         autoencoder_input_shape=full_field_shape[1:],
                         **config['model'])
loss_fn = "mse"#tf.keras.losses.get(config['training']['loss'])
model.compile(l_optimizer= tf.keras.optimizers.get(config['training']['l_optimizer']),
              loss=loss_fn,
              optimizer = tf.keras.optimizers.get(config['training']['ae_optimizer']),
              metrics = config['training'].get('metrics', None))
model.load_weights(weights_path)

In [ ]:
ds = iter(test_dataset)

In [ ]:
plt.rcParams['figure.dpi'] = 800
get_new=True
if get_new:
    (inp, target, norm_params), case_names = next(ds)
    pred = model(inp,autoencode=False)
    ae_pred = model(target,autoencode=True)

    targetn = normalizer.undo(target, norm_params[:,0,:])
    predn = normalizer.undo(pred, norm_params[:,0,:])
    ae_predn = normalizer.undo(ae_pred, norm_params[:,0,:])

idx=23
zsl=48

print(f"{case_names[idx].numpy().decode('utf-8')}")

vmax = 1.0
vmin = 0.0

stdf = 10.0
meann = tf.reduce_mean(target[idx,:,:,zsl])
stdn = tf.math.reduce_std(target[idx,:,:,zsl])
vmaxn = meann+stdf*stdn
vminn = meann-stdf*stdn

Lm = 7*20/40
coords = datasetf[case_names[idx].numpy()]['full_field_coords']
x = np.reshape(coords[:,:,zsl,0],[-1])
y = np.reshape(coords[:,:,zsl,1],[-1])

p = np.reshape(pred[idx,:,:,zsl,0], [-1])
pn = np.reshape(predn[idx,:,:,zsl,0], [-1])
ap = np.reshape(ae_pred[idx,:,:,zsl,0], [-1])
apn = np.reshape(ae_predn[idx,:,:,zsl,0], [-1])
t = np.reshape(target[idx,:,:,zsl,0], [-1])
tn = np.reshape(targetn[idx,:,:,zsl,0], [-1])

outer_polygon = [(-3,3), (-3,-3), (10,-3), (10,3)]
spatialmask = check_in_polygon(np.stack([x,y],-1), outer_polygon)
spatialmasked_indices = np.where(spatialmask)[0]
x = x[spatialmasked_indices]
y = y[spatialmasked_indices]
p = p[spatialmasked_indices]
pn = pn[spatialmasked_indices]
ap = ap[spatialmasked_indices]
apn = apn[spatialmasked_indices]
t = t[spatialmasked_indices]
tn = tn[spatialmasked_indices]

polygon_vertices = datasetf[case_names[idx].numpy()]['full_field_coords'][0,:,zsl]/Lm

#Normalized plots
plt.figure()
plt.tripcolor(x/Lm,y/Lm,p, shading='gouraud', cmap='rainbow', vmax = vmax, vmin = vmin)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

plt.figure()
plt.tripcolor(x/Lm,y/Lm,ap, shading='gouraud', cmap='rainbow', vmax = vmax, vmin = vmin)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

plt.figure()
plt.tripcolor(x/Lm,y/Lm,t, shading='gouraud', cmap='rainbow', vmax = vmax, vmin = vmin)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

#Unnormalized plots
plt.figure()
plt.tripcolor(x/Lm,y/Lm,pn, shading='gouraud', cmap='rainbow', vmax = vmaxn, vmin = vminn)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

plt.figure()
plt.tripcolor(x/Lm,y/Lm,apn, shading='gouraud', cmap='rainbow', vmax = vmaxn, vmin = vminn)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

plt.figure()
plt.tripcolor(x/Lm,y/Lm,tn, shading='gouraud', cmap='rainbow', vmax = vmaxn, vmin = vminn)
plt.fill(polygon_vertices[...,0], polygon_vertices[...,1], color='purple')
plt.colorbar()
plt.axis('equal')
plt.show()
plt.close()

print("==Slice % err==")
print(mape_with_threshold(p,t,pcterror_threshold=200))
print(mape_with_threshold(ap,t,pcterror_threshold=200))
print("==Whole snapshot % err==")
print(mape_with_threshold(pred[idx],target[idx],pcterror_threshold=200))
print(mape_with_threshold(ae_pred[idx],target[idx],pcterror_threshold=200))
print("==Slice % err unnormalized==")
print(mape_with_threshold(pn,tn,pcterror_threshold=200))
print(mape_with_threshold(apn,tn,pcterror_threshold=200))
print("==Whole snapshot % err unnormalized==")
print(mape_with_threshold(predn[idx],targetn[idx],pcterror_threshold=200))
print(mape_with_threshold(ae_predn[idx],targetn[idx],pcterror_threshold=200))

In [ ]:
#spanwise averaged error plots
plt.rcParams['figure.dpi'] = 800
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 18})
get_new=False
if get_new:
    (inp, target, norm_params), case_names = next(ds)
    pred = model(inp,autoencode=False)
    ae_pred = model(target,autoencode=True)

    targetn = normalizer.undo(target, norm_params[:,0,:])
    predn = normalizer.undo(pred, norm_params[:,0,:])
    ae_predn = normalizer.undo(ae_pred, norm_params[:,0,:])

coords = datasetf[case_names[idx].numpy()]['full_field_coords']
x = np.reshape(coords[:,:,zsl,0],[-1])
y = np.reshape(coords[:,:,zsl,1],[-1])

pn = np.reshape(predn[idx,:,:,zsl,0], [-1])
apn = np.reshape(ae_predn[idx,:,:,zsl,0], [-1])
tn = np.reshape(targetn[idx,:,:,zsl,0], [-1])

spanwise_avg_normalized_error = lambda X: np.reshape(np.mean(np.abs(X[idx,...,0] - targetn[idx,...,0]), -1), [-1]) / np.std(targetn[idx,...,0])

pnerr = spanwise_avg_normalized_error(predn)
apnerr = spanwise_avg_normalized_error(ae_predn)

outer_polygon = [(-3,3), (-3,-3), (10,-3), (10,3)]
spatialmask = check_in_polygon(np.stack([x,y],-1), outer_polygon)
spatialmasked_indices = np.where(spatialmask)[0]
x = x[spatialmasked_indices]
y = y[spatialmasked_indices]
pnerr = pnerr[spatialmasked_indices]
apnerr = apnerr[spatialmasked_indices]

polygon_vertices = datasetf[case_names[idx].numpy()]['full_field_coords'][0,:,zsl]

plt.figure()
plt.tripcolor(x/Lm, y/Lm, pnerr, shading='gouraud', cmap='Blues', vmax = 3.0, vmin = 0.0)
plt.fill(polygon_vertices[...,0]/Lm, polygon_vertices[...,1]/Lm, color='purple')
#plt.colorbar()
plt.axis('equal')
plt.xlabel(r"$x/L_m$")
plt.ylabel(r"$y/L_m$")
plt.savefig(f'/storage/paper/errormap_zavg_{expt_variable}.pdf', bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
from evtk import hl

def export_vtk(c, **fields):
    x = c[...,0].reshape(-1)
    y = c[...,1].reshape(-1)
    z = c[...,2].reshape(-1)
    data = {}
    for f in fields:
        data[f] = fields[f].reshape(-1).copy()
    hl.pointsToVTK("/storage/pressure3d", x, y, z, data = data)
    
export_vtk(coords, pred=predn[0,...,0], ae_pred=ae_predn[0,...,0].numpy(), gt = targetn[0,...,0].numpy())